## Introduction
The New York City Department of Health (DOH) inspects about 24,000 restaurants a year to monitor compliance with city and state regulations. They require restaurants to post their letter grades showing inspection results. Grades are assigned based on the following criteria:
* A: 0-13 points
* B: 14-27 points
* C: 28+ points

DOH has made this data publicly available. I will explore this data to see what insights I can uncover. This can be done in python by using pandas, or SQL by using joins and aggregate functions. Below it is performed in python using pandas.


In [1]:
import csv
import datetime as dt
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

log_file = "nyc_inspection_data%sWebExtract.txt"%os.sep
data = pd.io.parsers.read_csv(log_file,sep=',',quotechar='"',header=0,low_memory=False)
data.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225,7182875005,39,2014-07-01 00:00:00,F,06A,23,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,10B,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234,7184443838,27,2014-05-29 00:00:00,D,08C,10,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,02G,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531935 entries, 0 to 531934
Data columns (total 15 columns):
CAMIS           531935 non-null int64
DBA             531935 non-null object
BORO            531935 non-null int64
BUILDING        531286 non-null object
STREET          531934 non-null object
ZIPCODE         531933 non-null float64
PHONE           531874 non-null object
CUISINECODE     531935 non-null int64
INSPDATE        531935 non-null object
ACTION          531066 non-null object
VIOLCODE        520720 non-null object
SCORE           498411 non-null float64
CURRENTGRADE    234325 non-null object
GRADEDATE       232801 non-null object
RECORDDATE      531935 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 64.9+ MB


### Best zipcodes to eat in (from a food-safety perspective)
Let's examine all zipcodes with at least 100 inspections. I will return a tuple of (zipcode,mean score,standard err of score). I will list the top 10 zipcodes (Lower score is better)

In [3]:
topzips = data.ZIPCODE.value_counts()
sigtopzips = topzips[topzips>100]

#return [("11201", 21.9060928719313812, 0.179441607823702, 6762)] * 183
otuple=[]
for z in sigtopzips.index:
    zdat = data[data["ZIPCODE"]==z]["SCORE"]
    #some -1s and 0s exist in the data, remove them.
    zdat = zdat[zdat>=0]
    zmn = np.nanmean(zdat)
    n = len(zdat)-sum(np.isnan(zdat))  
    zse = np.nanstd(zdat) / np.sqrt(n)
    otuple.append(('%i' % z,zmn,zse,n))
sorted(otuple,key=lambda b: b[1])[:10]

[('10112', 15.268421052631579, 0.69216650772280208, 190),
 ('10121', 16.16, 0.98678467762729272, 100),
 ('10020', 17.607703281027103, 0.47052066565700235, 701),
 ('10282', 18.094972067039105, 1.3742492776818953, 179),
 ('11433', 18.104201680672269, 0.39341927874909449, 595),
 ('10307', 18.150990099009903, 0.58159252297705633, 404),
 ('11234', 18.945891783567134, 0.25599700582746587, 2495),
 ('10306', 19.015512465373963, 0.28278061489418643, 1805),
 ('11224', 19.079617834394906, 0.46489668359592956, 942),
 ('10044', 19.16030534351145, 1.10272013738915, 131)]

## Map this data
A list is nice, but not all that enlightening. So I will map this data. I will write this data out to a .csv file and import into [CartoDB](http://cartodb.com/) to produce a a map of average scores by zipcode.

In [4]:
with open('nyc_inspect_avg_zipcode.csv', "w") as the_file:
    csv.register_dialect("custom", delimiter=",")
    writer = csv.writer(the_file, dialect="custom")
    writer.writerows(otuple)    


Let us have a look.

In [5]:
from IPython.display import IFrame
IFrame('https://btquinn.cartodb.com/viz/3bcc03c6-1474-11e5-b379-0e4fddd5de28/embed_map',width=700,height=700)

Worst zipcode in each borough
* Manhattan: 10026(Central Park North/Harlem), 24.32
* Queens: 11354 (Flushing), 26.46
* Bronx: 10473 (Union Port), 25.97
* Brooklyn: 11220 (Sunset Park), 25.86
* Staten Island: 10303 (Arlington), 23.3

### Score by borough
I then looked at mean grade by borough. Here, I used the RI_Webextract_BigApps_Latest.xls file to see how to code the boroughs by name. There are five boroughs, but there is a sixth blank

In [6]:
otuple=[]
boro_names=["","MANHATTAN","THE BRONX","BROOKLYN","QUEENS","STATEN ISLAND"]
for b in np.arange(1,6):
    bdat = data[data["BORO"]==b]["SCORE"]
    #some -1s and 0s exist in the data, remove them.
    bdat = bdat[bdat>=0]
    bmn = np.nanmean(bdat)
    n = len(bdat)-sum(np.isnan(bdat))  
    bse = np.nanstd(bdat) / np.sqrt(n)
    otuple.append((boro_names[b],bmn,bse,n))
sorted(otuple,key=lambda b: b[1])  


[('STATEN ISLAND', 20.965332669446692, 0.1069902543813078, 16067),
 ('THE BRONX', 21.57383198069752, 0.07116924387758955, 45590),
 ('BROOKLYN', 22.181739480744341, 0.044390474694205227, 116667),
 ('MANHATTAN', 22.259562493258944, 0.033273874893593847, 203974),
 ('QUEENS', 22.687262825776894, 0.045650268113624298, 115685)]

There you have it. If you want a clean meal, go to Staten Island.

### Cuisine type
What are the least sanitary and most sanitary cuisine types? Place your bets now. I will calculate (cuisine, mean grade, stderr, number of inspections) for each cuisine type with at least 100 inspections.

Below you will see how I merge the cuisine data with the existing ratings data in pandas. This could be done in SQL using a JOIN and a few aggregate functions (AVG,STDDEV) and grouping by cuisine type.

In [7]:
# [("French", 21.9985734664764622, 0.177094690841052, 7010)] * 75
cuis_file = "nyc_inspection_data%sCuisine.txt"%os.sep
cuis_data = pd.io.parsers.read_csv(cuis_file,sep=',',quotechar='"',header=0,low_memory=False)
merg_data = pd.merge(data,cuis_data,on='CUISINECODE',sort=False)

cd_counts = merg_data.CODEDESC.value_counts()
top_cd = cd_counts[cd_counts>=100]
otuple=[]
for c in top_cd.index:
    cdat = merg_data[merg_data["CODEDESC"]==c]["SCORE"]
    cmn = np.nanmean(cdat)
    n = len(cdat)-sum(np.isnan(cdat))  
    cse = np.nanstd(cdat) / np.sqrt(n)
    otuple.append((c,cmn,cse,n))
sorted(otuple,key=lambda b: b[1])  


[('Hotdogs/Pretzels', 14.51923076923077, 0.99718007866706126, 156),
 ('Soups & Sandwiches', 14.961199294532628, 0.40904617986974662, 567),
 ('Ethiopian', 15.346153846153847, 0.42445782225432982, 286),
 ('Donuts', 15.536336731338375, 0.14351584576610082, 6082),
 ('Hotdogs', 15.5413870246085, 0.43889506978639414, 447),
 ('Ice Cream, Gelato, Yogurt, Ices',
  15.943380040034315,
  0.19519253686850047,
  3497),
 ('Sandwiches', 16.438442211055275, 0.14546661489127341, 6368),
 ('Juice, Smoothies, Fruit Salads',
  16.8989898989899,
  0.23828085890752101,
  2574),
 ('Caf\xe9/Coffee/Tea', 17.125518307681496, 0.10345270030595691, 14229),
 ('Sandwiches/Salads/Mixed Buffet',
  17.19119623655914,
  0.22068714968259523,
  2976),
 ('Cajun', 17.214285714285715, 0.58685422227236173, 168),
 ('Armenian', 17.233449477351915, 0.40830051859599903, 574),
 ('Other', 17.392756083757781, 0.36491771672206941, 1767),
 ('Hamburgers', 17.512352309344791, 0.13239562091401871, 7448),
 ('Bottled beverages, including wa

Surprised? Hot Dog vendors must run a tidy ship. Creole restaurants, on the other hand, have the hightest average. Shut those places down already.

### Which cuisines tend to have a disproportionate number of which violations?
This is a tricky question, as we can't simply count the violations. If we did, more popular cuisine types will tend to have more violations because they represent more restaurants. Additionally, some violation types are more common than others.

Therefore, to answer this question, we need to calculate the conditional probability of a specific violation type given a specific cuisine type and normalize it by the probability of the violation type. 

$$P\left( violation\,type  \,\middle|\,  cuisine\,type \right) \over P\left( violation\,type \right)$$

This ratio won't mean much when the number of violations is small, so I chose the cutoff of 100 violations of a violation type.

In [8]:
viol_file = "nyc_inspection_data%sViolation.txt"%os.sep
viol_data = pd.io.parsers.read_csv(viol_file,sep=',',quotechar='"',header=0,low_memory=False)

# Remove time from dates and convert to datetime format
merg_data['INSPDT'] = merg_data.INSPDATE.apply(lambda x: dt.datetime.strptime(x.split()[0],'%Y-%m-%d'))
viol_data['STRTDT'] = viol_data.STARTDATE.apply(lambda x: dt.datetime.strptime(x.split()[0],'%Y-%m-%d'))
viol_data['ENDDT'] = viol_data.ENDDATE.apply(lambda x: dt.datetime.strptime(x.split()[0],'%Y-%m-%d'))

viol_data.head(2)
viol_data.info()
merg_data.info()

             STARTDATE              ENDDATE CRITICALFLAG VIOLATIONCODE  \
0  1901-01-01 00:00:00  2003-03-23 00:00:00            Y           01A   
1  2003-03-24 00:00:00  2005-02-17 00:00:00            Y           01A   

                                       VIOLATIONDESC     STRTDT      ENDDT  
0  Current valid permit, registration or other au... 1901-01-01 2003-03-23  
1  Current valid permit, registration or other au... 2003-03-24 2005-02-17  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 719 entries, 0 to 718
Data columns (total 7 columns):
STARTDATE        719 non-null object
ENDDATE          719 non-null object
CRITICALFLAG     719 non-null object
VIOLATIONCODE    719 non-null object
VIOLATIONDESC    719 non-null object
STRTDT           719 non-null datetime64[ns]
ENDDT            719 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(5)
memory usage: 44.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 531935 entries, 0 to 531934
Data columns (total 17 column

In [9]:
datefields = ['STRTDT', 'ENDDT']
#Get the unique pairs of time windows
violdate_data = viol_data[datefields].drop_duplicates().reset_index(drop=True)
violdate_data['TIMEWINDOW']='A B C D E F G'.split()
violdate_data

,STRTDT,ENDDT,TIMEWINDOW
0,1901-01-01,2003-03-23,A
1,2003-03-24,2005-02-17,B
2,2005-02-18,2007-06-30,C
3,2007-07-01,2008-06-30,D
4,2008-07-01,2009-08-01,E
5,2009-08-02,2010-07-25,F
6,2010-07-26,2099-12-31,G


In [10]:
#merge the dataframe above with the whole violation data to fill in timewindow field
viol_data = pd.merge(viol_data,violdate_data,on=datefields,sort=False)
viol_data['VC_TW']=viol_data['VIOLATIONCODE']+viol_data['TIMEWINDOW']
viol_data.head(1)

,STARTDATE,ENDDATE,CRITICALFLAG,VIOLATIONCODE,VIOLATIONDESC,STRTDT,ENDDT,TIMEWINDOW,VC_TW
0,1901-01-01 00:00:00,2003-03-23 00:00:00,Y,01A,"Current valid permit, registration or other au...",1901-01-01,2003-03-23,A,01AA


In [11]:
#add timewindow field to merg_data. function maps date to timewindows 
def get_timewindow(date):
    if date.date() >= dt.date(2010,7,26):
        return 'G'
    elif date.date() >= dt.date(2009,8,2) and date.date() <= dt.date(2010,7,25):
        return 'F'
    elif date.date() >= dt.date(2008,7,1) and date.date() <= dt.date(2009,8,1):
        return 'E'
    elif date.date() >= dt.date(2007,7,1) and date.date() <= dt.date(2008,6,30):
        return 'D'
    elif date.date() >= dt.date(2005,2,18) and date.date() <= dt.date(2007,6,30):
        return 'C'
    elif date.date() >= dt.date(2003,3,24) and date.date() <= dt.date(2005,2,17):
        return 'B'
    elif date.date() <= dt.date(2003,3,23):
        return 'A'

merg_data['TIMEWINDOW']=merg_data['INSPDT'].apply(get_timewindow)
merg_data['VC_TW']=merg_data['VIOLCODE']+merg_data['TIMEWINDOW']
merg_data.head(1)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE,CODEDESC,INSPDT,TIMEWINDOW,VC_TW
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000,Bakery,2014-03-03,G,10FG


In [12]:
#Now we can merge both dataframes on the VC_TW field
viol_data_fields = ["VIOLATIONDESC","VC_TW"]
final_data = pd.merge(merg_data,viol_data[viol_data_fields],on='VC_TW',sort=False)
final_data.head(1)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE,CODEDESC,INSPDT,TIMEWINDOW,VC_TW,VIOLATIONDESC
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000,Bakery,2014-03-03,G,10FG,Non-food contact surface improperly constructe...


In [13]:
# Get counts of each violation description, and threshold at 100
cuis_viol_data = final_data.groupby('CODEDESC')['VIOLATIONDESC'].value_counts()
viol_counts = final_data['VIOLATIONDESC'].value_counts()
cuis_viol_thresh=cuis_viol_data[cuis_viol_data>100]

# Calculate the probabilities and ratio
total_viol = sum(cuis_viol_data)
result = []
for i,v in cuis_viol_thresh.iteritems():
    cv_cnt = cuis_viol_data[i]
    v_cnt = sum(cuis_viol_data[:,i[1]])
    c_cnt = sum(cuis_viol_data[i[0],])
    cp_vc = cv_cnt/(c_cnt+0.0)
    p_v = v_cnt/(total_viol+0.0)   
    result.append((i,cp_vc/p_v,cv_cnt))
sorted(result,key=lambda b: b[1],reverse=True)[:20]


[(('Japanese',
   'Food worker does not use proper utensil to eliminate bare hand contact with food that will not receive adequate additional heat treatment.'),
  3.2441736633039722,
  541),
 (('Caf\xe9/Coffee/Tea',
   '\x93Choking first aid\x94 poster not posted. \x93Alcohol and pregnancy\x94 warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.'),
  3.1528542316115793,
  175),
 (('Juice, Smoothies, Fruit Salads',
   'Food Protection Certificate not held by supervisor of food operations.'),
  3.0895762870547507,
  145),
 (('Donuts',
   'Accurate thermometer not provided in refrigerated or hot holding equipment.'),
  3.0373024983434593,
  130),
 (('Ice Cream, Gelato, Yogurt, Ices',
   'Food Protection Certificate not held by supervisor of food operations.'),
  2.9559491371480471,
  193),
 (('Thai', 'Thawing procedures improper.'), 2.6329943302133367, 151),
 (('Irish',
   '

### Conclusion
It appears the most common offense is roughly 'bare hand contact with food that will not receive adequate additional heat treatment' in Japanese restaurants. This is probably sushi handling. This particular violation occurs 3 times more in Japanese restaurants than in all restaurants combined.

Towards the bottom of the list is one of the more offensive violations: *Filth flies*. It is much more likely to occur in Hamburger joints than other restaurants. There were 600 cited violations of this code for Hamburger restaurants!

Bon appetit.